In [1]:
import drawSvg as draw
from enum import Enum
import math
strip =[]
stripwidth = 10
def render (strip, stripwidth):
    striplength = 0
    for element in strip:
        striplength += element.get_length()
    d = draw.Drawing(striplength,stripwidth)
    d.append(draw.Rectangle(0, 0, striplength, stripwidth,stroke='red', fill='none'))        
    d.saveSvg('test.svg')
    d.rasterize()
    return d

In [2]:
class Plain:
    def __init__(self, length):
        self.length = length
    def get_length(self,stripwidth=10):
        return self.length
def add_plain(strip,len):
    strip.append(Plain(len))
    return strip

strip = add_plain(strip,100)
render (strip, stripwidth)

In [3]:
class Fold:
    def __init__(self, mv, angle):
        self.mv = mv
        self.angle = angle
    def get_length(self,stripwidth):
        return stripwidth*math.tan(math.radians(90-self.angle))
class MV(Enum):
    MOUNTAIN=0
    VALLEY=1 
def add_fold(strip,mv,angle):
    strip.append(Fold(mv,angle))
    return strip
def render (strip, stripwidth):
    striplength=0
    for element in strip: striplength += abs(element.get_length(stripwidth))   
    d = draw.Drawing(striplength,stripwidth)
    x = 0
    for element in strip:
        if type(element) is Plain:
            x +=element.length
        if type(element) is Fold:
            l = element.get_length(stripwidth)    
            x1,x2 = x, x+abs(l) 
            if l <0: x1,x2 = x2,x1
            d.append(draw.Line(x1,0,x2,stripwidth,stroke='red' if element.mv is MV.MOUNTAIN else 'blue', fill='none')) 
            x+= abs(l)
    d.append(draw.Rectangle(0, 0, striplength, stripwidth,stroke='red', fill='none'))        
    d.saveSvg('test.svg')
    d.rasterize()
    return d

strip = add_fold(strip,MV.MOUNTAIN, -45)
strip = add_fold(strip,MV.MOUNTAIN, 45)
strip = add_plain(strip,100)
strip = add_fold(strip,MV.VALLEY, -30)

render (strip, stripwidth)

In [4]:
def add_corner_gadget (strip,mv):
    strip = add_fold(strip,mv,45)
    strip = add_fold(strip,mv,-45)
    return strip
strip = add_corner_gadget(strip,MV.MOUNTAIN)
render (strip, stripwidth)

In [5]:
class Direction(Enum):
    L=-1
    R=1 
def add_corner_gadget (strip,mv,d):
    strip = add_fold(strip,mv,45*d.value)
    strip = add_fold(strip,mv,-45*d.value)
    return strip
strip = add_corner_gadget(strip,MV.MOUNTAIN,Direction.R)
strip = add_plain(strip,100)
strip = add_corner_gadget(strip,MV.MOUNTAIN,Direction.L)
render (strip,stripwidth)

In [6]:
import numpy

def get_distance(a,b):
    return numpy.linalg.norm(numpy.array(a) - numpy.array(b))
def cosinelaw(adj_side1,adj_side2,opp_side):
    return math.acos((adj_side1 ** 2 + adj_side2 ** 2 - opp_side **2) / (2*adj_side1 * adj_side2))
def culatulate_angles(a,b,c):
    ab = get_distance(a,b)
    bc = get_distance(b,c)
    ac = get_distance(a,c)
    alpha = cosinelaw(ab,ac,bc)
    beta = cosinelaw(ab,bc,ac)
    gamma = cosinelaw(ac,bc,ab)
    return alpha,beta,gamma
def corner_extension(gamma,beta,direction,strip_width):
    return math.tan(0.5*math.pi - (gamma if direction is Direction.L else beta))*strip_width
def add_triangle_gadget (strip,mv,strip_width,t):
    a,b,c = t[0],t[1],t[2]
    alpha,beta,gamma = culatulate_angles(a,b,c)
    height = get_distance(a,b) * math.sin(beta)
    d = Direction.R
    strip = add_plain(strip, corner_extension(gamma, beta,d,strip_width)) # start with a plain stip
    strip = add_corner_gadget(strip,mv,d)  # fold a corner
    y=0
    while(y < height):
        d = Direction.R if d is Direction.L else Direction.L
        strip = add_plain(strip,(get_distance(b,c)*y)/height + corner_extension(gamma,beta,d,strip_width))
        strip = add_corner_gadget(strip,mv,d)
        y+=strip_width
    return strip
strip = []
stripwidth = 10
strip = add_triangle_gadget(strip,MV.MOUNTAIN,stripwidth,[[50,-20,200],[50,-20,0],[50,180,0]]) 
render (strip, stripwidth)

In [7]:
import openmesh as om

mesh = om.TriMesh()
mesh = om.read_trimesh("cube.stl")
strip = []
stripwidth = 4
for fh in mesh.faces():
    triangle = []
    [triangle.append(mesh.point(vertex)) for vertex in mesh.fv(fh)]
    strip = add_triangle_gadget(strip,MV.MOUNTAIN if fh.idx()%2==0 else MV.VALLEY,stripwidth,triangle)

render (strip, stripwidth)     